In [3]:
# 9/25/19 Requirement   Group 24

#      We have shrunk a little, and now have just 28 teams, so I'm
# changing the experimental design structure a little.  The next
# assignment is due Thursday, Oct. 10.

#      There will be no class on Monday, Oct. 7, since Duke is on Fall Break.

#       The RA asks that you report your results using the attached
# template.  I also attach an updated spreadsheet for the teams.  If
# anyone is still in a singleton team, please let me know and I shall
# match you up with a team from your university.

#       The assignment is to train with four hidden layers, each having
# 1,024 nodes.  The activation function will be ReLU for the top two
# layers, and linear for the bottom two layers (these lowest layer
# receives the inputs).  We shall consider four datasets: MNIST fashion,
# Cifar-10, cat/non-cat, and street view of digits. All groups will train
# with 1,000 minibatches of size 128 for a total of 128,000 observations.

#       The experiment will compare CNNs to regular NNs.  Teams doing CNNs
# should use the kernel shown on slide 4 of dl6.pdf (which will be posted
# later today).  You will also want to use zero padding, as discussed on
# slide 14.

# If your team's row number in the spreadsheet ends with a 1, use regular
# a regular NN with the MNIST fashion data.

# If it ends with a 2, use regular NN with the Cifar-10 data.

# If it ends with a 3, use regular NN with the cat/non-cat data.

# If 4, use regular NN with the street view of digits data.

# If 5, use CNN with the MNIST fashion data.

# If 6, use CNN with Cifar-10.

# If 7, use CNN with cat/non-cat.

# If 8, use CNN with streetview.

# If 9, use CNN with Cifar-10, but use linear activation in only the first
# hidden layer, and ReLU in the top three hidden layers.

# If 0, use CNN with Cifar-10, but have five hidden layers, such that the
# bottom two are linear and the top three are ReLU.



In [1]:
#set up seed
#https://machinelearningmastery.com/reproducible-results-neural-networks-keras/
from numpy.random import seed
seed(23)
from tensorflow import set_random_seed
set_random_seed(34)
import numpy as np

/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
#       The assignment is to train with four hidden layers, each having
# 1,024 nodes.  The activation function will be ReLU for the top two
# layers, and linear for the bottom two layers (these lowest layer
# receives the inputs).  We shall consider four datasets: MNIST fashion,
# Cifar-10, cat/non-cat, and street view of digits. All groups will train
# with 1,000 minibatches of size 128 for a total of 128,000 observations.

# If 4, use regular NN with the street view of digits data.

# Street View House Numbers (SVHN) Dataset
# http://ufldl.stanford.edu/housenumbers/
# import from SciPy

import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt


%matplotlib inline

train_data=sio.loadmat('train_32x32.mat')
test_data=sio.loadmat('test_32x32.mat')
extra_data=sio.loadmat('extra_32x32.mat')

x_train = train_data['X']
y_train = train_data['y']
x_test = test_data['X']
y_test = test_data['y']
x_extra = extra_data['X']
y_extra = extra_data['y']


print(x_train.shape)
print(x_extra.shape)
print(y_train.shape)
print(y_extra.shape)
# Use extra data as train data
x_train = np.concatenate([x_train, x_extra],axis=3)
y_train = np.concatenate([y_train, y_extra])
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(np.unique(y_test))



x_train_1=x_train.reshape((604388, 32*32*3))
print(x_train_1.shape)

x_test_1=x_test.reshape((26032, 32*32*3))
print(x_test_1.shape)

import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

# define Keras model
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.layers import Flatten


x_train_1=x_train_1.astype('float32') / 255
x_test_1=x_test_1.astype('float32') / 255
y_train_m1=y_train
y_train_m1[y_train_m1==10]=0
y_test_m1=y_test
y_test_m1[y_test_m1==10]=0
y_train_fla=y_train_m1
y_train_cat=to_categorical(y_train_fla)
y_test_fla=y_test_m1
y_test_cat=to_categorical(y_test_fla)

from random import choices
num=x_train_1.shape[0]
a=range(num)
print(a)
#idx=choices(a, k=128000)
idx=np.random.choice(a,128000,replace=False)
print(len(np.unique(idx)))
print(len(idx))

x_train_final=x_train_1[idx,]
y_train_final=y_train_cat[idx,]


model = Sequential()
model.add(Dense(1024, activation='relu',input_shape=(32 * 32 * 3,)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='linear'))
model.add(Dense(1024, activation='linear'))
model.add(Dense(10, activation='softmax'))

# compile keras model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# model.fit(x_train_1, y_train_cat, batch_size=128, epochs=1)
model.fit(x_train_final, y_train_final, batch_size=128, epochs=2)
# results = model.evaluate(x_test_1, y_test_cat)

test_loss, test_acc = model.evaluate(x_test_1, y_test_cat)
print('test_acc:', test_acc)
print('test_loss:', test_loss)

#weights for 1st hidden layer
print('Weights for 1st hidden layer')
print(model.layers[0].get_weights()[0])
#intercept for 1st hidden layer
print("Intercept for 1st hidden layer")
print(model.layers[0].get_weights()[1])

#weights for 2nd hidden layer
print('Weights for 2nd hidden layer')
print(model.layers[1].get_weights()[0])
#intercept for 2nd hidden layer
print("Intercept for 1st hidden layer")
print(model.layers[1].get_weights()[1])

#weights for 3rd hidden layer
print('Weights for 1st hidden layer')
print(model.layers[2].get_weights()[0])
#intercept for 1st hidden layer
print("Intercept for 1st hidden layer")
print(model.layers[2].get_weights()[1])

#weights for 4th hidden layer
print('Weights for 2nd hidden layer')
print(model.layers[3].get_weights()[0])
#intercept for 2nd hidden layer
print("Intercept for 1st hidden layer")
print(model.layers[3].get_weights()[1])

#weights for output layer
print('Weights for output layer')
print(model.layers[4].get_weights()[0])
#intercept for output layer
print("Intercept for output layer")
print(model.layers[4].get_weights()[1])

(32, 32, 3, 73257)
(32, 32, 3, 531131)
(73257, 1)
(531131, 1)
(32, 32, 3, 604388)
(604388, 1)
(32, 32, 3, 26032)
(26032, 1)
[ 1  2  3  4  5  6  7  8  9 10]
(604388, 3072)
(26032, 3072)


Using TensorFlow backend.


range(0, 604388)
128000
128000





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/2
128000/128000 [==============================] - 132s 1ms/step - loss: 13.3340 - acc: 0.1719
Epoch 2/2
26032/26032 [==============================] - 5s 195us/step
test_acc: 0.1958743085433313
test_loss: 12.960974645819778
Weights for 1st hidden layer
[[ 0.03287894 -0.01961638 -0.00676601 ... -0.02203188 -0.00179878
  -0.03352737]
 [ 0.02568968 -0.03190538  0.0069748  ...  0.03534507 -0.02867789
  -0.01155834]
 [ 0.03051473  0.03034975  0.00331167 ... -0.01055502 -0.02858113
  -0.026676  ]
 ...
 [-0.00781     0.03239557  0.01158051 ...  0.01239249 -0.02546391
  -0.02103624]
 [ 0.00087753  0.03487987  0.03365701 ... -0.01658826  0.03081007
  -0.01742562]
 [-0.03488891  0.03067526  0.02412577 ... -0.03101038  0.01185026
  -0.02036925]]
Intercept for 1st hidden layer
[-0.0031609  -0.00316189  0.00316057 ...  0.00316135  0.00314881
 -0.00316203]
Weigh